In [1]:
from flask import Flask, request, send_file
import base64
import numpy as np
import tensorflow
import keras
from keras.preprocessing import image
from keras.models import load_model
from flask_ngrok import run_with_ngrok
from PIL import ImageTk, Image
import fpdf
from datetime import date
import tensorflow as tf

Using TensorFlow backend.


In [3]:
print(tf.__version__)

1.15.0


In [9]:
def fetch_results(image_str,patient_name):
  with tf.name_scope("predict"):
    loaded_graph = tf.Graph()
    with tf.Session(graph=loaded_graph) as sess:
      model = tensorflow.keras.models.load_model("./minor-server/content/keras_model.h5")
      with open("./minor-server/content/{}.jpeg".format(patient_name) , "wb") as image_file_recv:
        image_file_recv.write(base64.b64decode(image_str.encode('utf-8')))
      testing_image = image.load_img("./minor-server/content/{}.jpeg".format(patient_name),target_size = (224, 224))
      testing_image = image.img_to_array(testing_image)
      testing_image = np.expand_dims(testing_image, axis = 0)
      pdf = fpdf.FPDF(format ='letter')
      pdf.add_page()
      pdf.set_fill_color(230,230,0)
      pdf.image("./minor-server/content/Banner.png",0,0,215,50,'PNG')
      pdf.set_font("Arial", size = 12)
      pdf.ln(50)
      pdf.set_font("Arial","B", size = 12)
      pdf.multi_cell(0, 10, txt="{}".format(date.today()), border=0, align="R",fill=False)
      pdf.multi_cell(0, 10, txt="{}".format(patient_name.upper()), border=0, align="L",fill=False)
      pdf.set_font("Arial", size = 12)
      pdf.ln(90)
      pdf.set_font("Arial","B", size = 9)
      pdf.image("./minor-server/content/{}.jpeg".format(patient_name),75,90,75,75,'JPEG')
      pdf.multi_cell(200, 3, txt="Patient's Retina Picture", border=0, align="C",fill=False)
      pdf.ln(10)

      pred = model.predict(testing_image)

      is_cancer = False
      prob = 0.0
      pdf.set_font("Arial","B", size = 12)
      pdf.multi_cell(200, 10, txt="Findings in diagnosis:", border=0, align="L",fill=False)
      if pred[0][0] > pred[0][1]:
        is_cancer = True
        prob = pred[0][0]*100
        pdf.set_font("Arial", size = 12)
        pdf.multi_cell(200, 10, txt="Cancer cells were found during the diagnosis. This findings lead to the conclusion that diabetic retinopathy disease is present.", border=0, align="L",fill=True)
      else:
        prob = pred[0][1]*100
        pdf.multi_cell(200, 10, txt="No presence of Cancerous Cells was found. If the problem still persists you please consult a doctor.", border=0, align="L",fill=True)

      pdf.ln(20)
      pdf.set_font("Arial","B", size = 12)
      pdf.multi_cell(0, 10, txt="****This is a computer generated report. This report is only to guide for further investigation, no action should be taken based on this report.****", border=0, align="C",fill=False)
      pdf.output("./minor-server/content/{}.pdf".format(patient_name))

      return {"is_cancer" : is_cancer, "prob" : prob}

In [5]:
app = Flask(__name__)

# run_with_ngrok(app)

@app.route("/detect" , methods = ['POST'])
def detect():
  # try:
  if request.method == 'POST':
    return fetch_results(request.form.get('image_str'),request.form.get('patient_name'))
  # except:
  #   return "ERROR OCCURED", 500

@app.route("/getReport", methods = ['GET'])
def getReport():
  try:
    if request.method == 'GET':
      return send_file("./content/{}.pdf".format(request.args.get('patient_name')), attachment_filename = "{}_DR_Report.pdf".format(request.args.get('patient_name')))
  except:
    return "ERROR", 500

In [ ]:
app.run()

In [6]:
with open("./minor-server/content/test_data/sym/1032_left.jpeg", "rb") as image_file:
  encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
#   print(fetch_results(encoded_string,"lol"))

In [7]:
!pwd
testing_image = image.load_img("./minor-server/content/test_data/sym/1032_left.jpeg",target_size = (224, 224))
testing_image = image.img_to_array(testing_image)
testing_image = np.expand_dims(testing_image, axis = 0)

# print(model.predict(testing_image))

/home/darksun27/temp/Minor


In [10]:
print(fetch_results(encoded_string,"Siddharth"))

{'is_cancer': True, 'prob': 85.18055081367493}


In [2]:
model = tensorflow.keras.models.load_model("./minor-server/content/keras_model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
